# Adicionando e Removendo Dados

## Sobre os Dados
Neste notebook, estaremos trabalhando com dados de terremotos de 18 de setembro de 2018 a 13 de outubro de 2018 (obtidos do Serviço Geológico dos Estados Unidos (USGS) usando o [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/)).

## Configuração
Vamos trabalhar novamente com o arquivo `data/earthquakes.csv`, então precisamos importar e lê-lo.

In [1]:
import pandas as pd

df = pd.read_csv(
    'data/earthquakes.csv',
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

## Criando novos dados
### Adicionando novas colunas
Novas colunas são adicionadas à direita das colunas originais e podem ser um único valor, que será **transmitido** ao longo das linhas do dataframe:

In [2]:
df['source'] = 'USGS API'
df.head()

,alert,mag,magType,place,time,title,tsunami,source
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API


...ou uma máscara booleana:

In [3]:
df['mag_negative'] = df.mag < 0
df.head()

,alert,mag,magType,place,time,title,tsunami,source,mag_negative
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,USGS API,False
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,USGS API,False
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,USGS API,False


#### Adicionando a coluna `parsed_place`
Temos um problema de reconhecimento de entidade com a coluna `place`. Existem várias entidades que têm múltiplos nomes nos dados (por exemplo, CA e California, NV e Nevada).

In [4]:
df.place.str.extract(r', (.*$)')[0].sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Australia', 'Azerbaijan', 'B.C., MX', 'Barbuda', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba ', 'British Virgin Islands',
       'Burma', 'CA', 'California', 'Canada', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Ecuador region',
       'El Salvador', 'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala',
       'Haiti', 'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Italy', 'Jamaica', 'Japan', 'Kansas',
       'Kentucky', 'Kyrgyzstan', 'Martinique', 'Mauritius', 'Mayotte',
       'Mexico', 'Missouri', 'Montana', 'NV', 'Nevada', 'New Caledonia',
       'New Hampshire', 'New Mexico', 'New Zealand', 'Nicaragua',
       'North Carolina', 'Northern Mariana Islands', 'Oklahoma', 'Oregon',
       'Pakistan', 'Papua New Guinea', 'Peru', 'Philippines',
       'Puerto Rico', 'Roman

Substituir partes dos nomes de `place` para atender às nossas necessidades:

In [5]:
df['parsed_place'] = df.place.str.replace(
    r'.* of ', '', regex=True  # remove anything saying <something> of <something>
).str.replace(
    r'.* and ', '', regex=True  # remove anything saying <something> and <something>
).str.replace(
    'the ', ''  # remove "the "
).str.replace(
    r'CA$', 'California', regex=True  # fix California
).str.replace(
    r'NV$', 'Nevada', regex=True  # fix Nevada
).str.replace(
    r'MX$', 'Mexico', regex=True  # fix Mexico
).str.replace(
    r' region$', '', regex=True  # chop off endings with " region"
).str.replace(
    'northern ', ''  # remove "northern "
).str.replace(
    'Fiji Islands', 'Fiji'  # line up the Fiji places
).str.replace(
    r'^.*, ', '', regex=True  # remove anything else extraneous from the beginning
).str.strip()  # remove any extra spaces

Agora podemos usar um único nome para obter todos os terremotos para esse lugar (embora isso ainda não seja perfeito):

In [6]:
df.parsed_place.sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Ascension Island', 'Australia', 'Azerbaijan', 'Balleny Islands',
       'Barbuda', 'Bolivia', 'British Virgin Islands', 'Burma',
       'California', 'Canada', 'Carlsberg Ridge',
       'Central East Pacific Rise', 'Central Mid-Atlantic Ridge', 'Chile',
       'China', 'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'El Salvador',
       'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala', 'Haiti',
       'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indian Ocean Triple Junction', 'Indonesia', 'Iran', 'Iraq',
       'Italy', 'Jamaica', 'Japan', 'Kansas', 'Kentucky',
       'Kermadec Islands', 'Kuril Islands', 'Kyrgyzstan', 'Martinique',
       'Mauritius', 'Mayotte', 'Mexico', 'Mid-Indian Ridge', 'Missouri',
       'Montana', 'Nevada', 'New Caledonia', 'New Hampshire',
       'New Mexico', 'New Zealand', 'Nicaragua', 'North Carolina',


#### Usando o método `assign()` para criar colunas
Para criar várias colunas de uma vez ou atualizar colunas existentes, podemos usar `assign()`:

In [7]:
df.assign(
    in_ca=df.parsed_place.str.endswith('California'),
    in_alaska=df.parsed_place.str.endswith('Alaska')
).sample(5, random_state=0)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False


Com o uso de funções `lambda`, o método `assign()` se torna ainda mais poderoso. **Funções lambda** são funções anônimas geralmente definidas em uma linha e para uso único. O método `assign()` passa o dataframe inteiro para a função `lambda` como `x`; a partir daí, podemos selecionar as colunas `in_ca` e `in_alaska`, que estão sendo criadas na mesma chamada para `assign()`. Aqui, usamos uma função `lambda` para criar uma nova coluna, `neither`, que indica se o terremoto não foi nem no Alasca nem na Califórnia:

In [8]:
df.assign(
    in_ca=df.parsed_place == 'California',
    in_alaska=df.parsed_place == 'Alaska',
    neither=lambda x: ~x.in_ca & ~x.in_alaska
).sample(5, random_state=0)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,in_ca,in_alaska,neither
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua",1537749595210,"M 4.8 - 73km SSW of Masachapa, Nicaragua",0,USGS API,False,Nicaragua,False,False,True
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington",1538227540460,"M 1.1 - 28km NNW of Packwood, Washington",0,USGS API,False,Washington,False,False,True
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska",1538259609862,"M 1.8 - 77km SSW of Kaktovik, Alaska",0,USGS API,False,Alaska,False,True,False
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska",1538464751822,"M 1.5 - 102km NW of Arctic Village, Alaska",0,USGS API,False,Alaska,False,True,False
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA",1538801713880,"M 0.9 - 26km ENE of Pine Valley, CA",0,USGS API,False,California,True,False,False


#### Concatenação
Digamos que estivéssemos trabalhando com dois dataframes separados, um com terremotos acompanhados por tsunamis e outro com terremotos sem tsunamis. Se quisermos analisar os terremotos como um todo, gostaríamos de concatenar os dataframes em um único:

In [9]:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]

tsunami.shape, no_tsunami.shape

((61, 10), (9271, 10))

Concatenar ao longo do eixo das linhas (`axis=0`) é equivalente a adicionar na parte inferior. Ao concatenar nossos terremotos com tsunamis e aqueles sem tsunamis, obtemos de volta o conjunto completo de dados de terremotos:

In [10]:
pd.concat([tsunami, no_tsunami]).shape

(9332, 10)

Nós estávamos trabalhando com um subconjunto das colunas do arquivo CSV, mas suponha que agora queremos obter algumas das colunas que ignoramos quando lemos os dados. Como adicionamos novas colunas neste notebook, não queremos ler o arquivo novamente e realizar essas operações. Em vez disso, vamos concatenar ao longo das colunas (`axis=1`) para adicionar de volta o que estamos perdendo:

In [11]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389218,",-480.0
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California,NaN,",ci37389202,",-480.0


Observe o que acontece se os índices não estiverem alinhados:

In [12]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.4 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.3 - 9km NE of Aguanga, CA",0.0,USGS API,False,California,NaN,NaN,NaN
1539475168010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389218,",-480.0
1539475129610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389202,",-480.0


Digamos que queremos unir os dataframes `tsunami` e `no_tsunami`, mas o dataframe `no_tsunami` tem uma coluna adicional. O parâmetro `join` especifica como lidar com qualquer sobreposição de nomes de coluna (quando anexados na parte inferior) ou de nomes de linha (quando concatenados à esquerda/direita). Por padrão, isso é `outer`, então mantemos tudo; no entanto, se usarmos `inner`, manteremos apenas o que é comum:

In [13]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner'
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place
36,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island
118,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California


Além disso, usamos `ignore_index`, já que o índice não significa nada para nós aqui. Isso nos dá valores sequenciais em vez do que tínhamos no resultado anterior:

In [14]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

,alert,mag,magType,place,time,title,tsunami,source,mag_negative,parsed_place
0,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,USGS API,False,Christmas Island
1,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,USGS API,False,Russia
2,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,USGS API,False,California
3,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,USGS API,False,California


## Excluindo Dados Indesejados
Colunas podem ser excluídas usando a sintaxe de dicionário com `del`:

In [15]:
del df['source']
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'mag_negative', 'parsed_place'],
      dtype='object')

Se não sabemos se a coluna existe, devemos usar um bloco `try`/`except`:

In [16]:
try:
    del df['source']
except KeyError:
    print('Não foi possível deletar a coluna source')

Não foi possível deletar a coluna source


Também podemos usar `pop()`.

In [17]:
mag_negative = df.pop('mag_negative')
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'parsed_place'],
      dtype='object')

Note que agora temos uma máscara em `mag_negative`:

In [18]:
mag_negative.value_counts()

mag_negative
False    8841
True      491
Name: count, dtype: int64

Agora, podemos usar `mag_negative` para filtrar nossos dados:

In [19]:
df[mag_negative].head()

,alert,mag,magType,place,time,title,tsunami,parsed_place
39,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539458844506,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
49,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539455017464,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
135,NaN,-0.40,ml,"10km SSE of Beatty, Nevada",1539422175717,"M -0.4 - 10km SSE of Beatty, Nevada",0,Nevada
161,NaN,-0.02,md,"20km SSE of Ronan, Montana",1539412475360,"M -0.0 - 20km SSE of Ronan, Montana",0,Montana
198,NaN,-0.20,ml,"60km N of Pahrump, Nevada",1539398340822,"M -0.2 - 60km N of Pahrump, Nevada",0,Nevada


### Usando o método `drop()`
Podemos excluir linhas passando uma lista de índices para o método `drop()`. Observe no exemplo a seguir que, ao solicitar as primeiras 2 linhas com `head()`, obtemos a 3ª e 4ª linhas porque excluímos as duas primeiras originais com `drop([0, 1])`:

In [20]:
df.drop([0, 1]).head(2)

,alert,mag,magType,place,time,title,tsunami,parsed_place
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,California


O método `drop()` elimina ao longo do eixo das linhas por padrão. Se passarmos uma lista de colunas com o argumento `columns`, podemos excluir colunas:

In [21]:
cols_to_drop = [
    col for col in df.columns
    if col not in ['alert', 'mag', 'title', 'time', 'tsunami']
]
df.drop(columns=cols_to_drop).head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


Também temos a opção de usar `axis=1`:

In [22]:
df.drop(columns=cols_to_drop).equals(
    df.drop(cols_to_drop, axis=1)
)

True

Por padrão, o método `drop()`, junto com a maioria dos métodos de `DataFrame`, retorna um novo objeto `DataFrame`. Se quisermos modificar diretamente o objeto com o qual estamos trabalhando, podemos passar `inplace=True`. Isso deve ser usado com cuidado:

In [23]:
df.drop(columns=cols_to_drop, inplace=True)
df.head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


<hr>
<div>
    <a href="./5-subsetting_data.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="../ch_03/1-wide_vs_long.ipynb">
        <button style="float: right;">Chapter 3 &#8594;</button>
    </a>
</div>
<br>
<hr>